In [1]:
import sys
import talib as talib
import pandas as pd
import numpy as np
from pyecharts import options as opts
sys.path.append("..") 
from imp import reload
from src.ts_stock import TsStock
from src.chart import Chart
from src.strategy import FirstStrategy
from src.sina_stock import SinaStock


tsStock = TsStock()
sinaStock = SinaStock()
chart = Chart();


/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [9]:
sma_periods = [8,17,25,43,99,145,318,453]
keyword = '002019'
init_amount = 100000

stock_info = tsStock.get_stock_info(keyword)
stragety = FirstStrategy(tsStock, keyword , start_date="2010-01-01",sma_periods = sma_periods,sina_realtime=sinaStock)
df = stragety.simulate(start_date="20170101",
                           init_amount=init_amount,
                       max_add_count=4,
                       stop_price_factor=4
                       
                       )

In [10]:
line = chart.get_line_ba(df,init_amount,stock_info)
line.render_notebook()


In [11]:

bs = {}
bs['buy'] = df.loc[df['action']=='buy',['trade_date','unit_price']]
bs['sell'] = df.loc[df['action'] == 'sell', ['trade_date','unit_price']]
df_stop_price = df.loc[df['unit_account']>0,['trade_date','stop_price']]
trade_dates = np.array(df.trade_date)
kdatas = np.array(df.loc[:,['open','close','low','high']])
vols = np.array(df['vol'])
df['bol_up'], df['bol_mi'], df['bol_lo'] = talib.BBANDS(df.close, 20, 2, 2)
sma_dic = {}
for period in sma_periods:
    name = "SMA" + str(period)
#     sma = talib.SMA(np.array(df.close),period)
#     df[name] =sma;
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] =np.array(df[name]).round(2).tolist();


grid_chart= chart.get_grid(
    stock_info,
    trade_dates.tolist(),
    kdatas.tolist(),
    vols.tolist(),
    ma_dic=sma_dic,
    bs = bs, 
    df_stop_price=df_stop_price,
    bol=df[['trade_date','bol_up','bol_mi','bol_lo']] 
)



turnover_chart = chart.get_turnover_chart(df)

adosc_chart = chart.get_adosc_chart(df)

grid_chart.add(
    turnover_chart,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="68%", height="7%"
    ),
)
grid_chart.add(
    adosc_chart,
    grid_opts=opts.GridOpts(
        pos_left="6%", pos_right="8%", pos_top="75%", height="7%"
    ),
)

grid_chart.render_notebook()


In [12]:
df.tail(10)[['close','high','add_price','stop_price','unit','unit_price','R','P/R','ATR','signal','pre_buy_count']]

,close,high,add_price,stop_price,unit,unit_price,R,P/R,ATR,signal,pre_buy_count
trade_date,,,,,,,,,,,
2019-10-10,13.11,13.17,12.051545,12.300749,0.0,0.0,1.656453,0.995523,0.452313,pre_buy,4.0
2019-10-11,13.80,13.91,12.058698,12.243519,0.0,0.0,1.656453,1.412075,0.466620,pre_buy,5.0
2019-10-14,13.99,14.10,12.057366,12.254179,0.0,0.0,1.656453,1.526778,0.463955,pre_buy,6.0
2019-10-15,13.83,13.95,12.053487,12.285212,0.0,0.0,1.656453,1.430186,0.456197,pre_buy,7.0
2019-10-16,14.24,14.47,12.057963,12.609403,0.0,0.0,1.656453,1.677703,0.465149,pre_buy,8.0
2019-10-17,14.40,14.41,12.054860,12.634227,0.0,0.0,1.656453,1.774295,0.458943,pre_buy,9.0
2019-10-18,13.51,14.56,12.068681,12.613658,0.0,0.0,1.656453,1.237002,0.486586,,0.0
2019-10-21,13.36,13.86,12.075949,12.555512,0.0,0.0,1.656453,1.146448,0.501122,,0.0
2019-10-22,14.70,14.70,12.100727,12.497291,0.0,0.0,1.656453,1.955405,0.550677,pre_buy,1.0


In [6]:
df.to_excel('strategy_detail.xlsx')

In [7]:
df.tail(10).to_excel('strategy_detail.xlsx')

In [8]:
75.36 - 0.5 * 2.046


74.337